In [1]:
import geopandas as gpd
import pandas as pd
import folium
import re
from folium.plugins import MarkerCluster
import os
from folium.plugins import TimestampedGeoJson

In [2]:
def load_data():
    # DETER alerts_df_df_df (GEODF)
    alerts_df = gpd.read_file('../data/deter-amz-public-2024out08/deter-amz-deter-public.shp', encoding='utf-8')
    alerts_df.loc[alerts_df['CLASSNAME'] == 'DEGRDACAO', 'CLASSNAME'] = 'DEGRADACAO'
    alerts_df = alerts_df[~(alerts_df['CLASSNAME'] == 'CORTE_SELETIVO')]
    alerts_df['VIEW_DATE'] = pd.to_datetime(alerts_df['VIEW_DATE'])
    alerts_df['ANO'] = alerts_df['VIEW_DATE'].dt.year
    alerts_df['MES'] = alerts_df['VIEW_DATE'].dt.month
    alerts_df['MES/ANO'] = alerts_df['VIEW_DATE'].dt.strftime('%Y-%m')

    # DETER ALERTS (CSV)
    df_deter = pd.DataFrame(alerts_df)
    df_deter = df_deter.drop(columns=['FID', 'QUADRANT', 'PATH_ROW', 'SENSOR', 'SATELLITE', 'geometry'])

    
    #IBGE DATA
    legal_amazon = gpd.read_file('../data/brazilian_legal_amazon/brazilian_legal_amazon.shp',encoding='utf-8')
    states = gpd.read_file('../data/states_legal_amazon/states_legal_amazon.shp',encoding='utf-8')
    
    ac = gpd.read_file('../data/malhas_regionais_ibge/AC_Municipios_2022/AC_Municipios_2022.shp', encoding='utf-8')
    am = gpd.read_file('../data/malhas_regionais_ibge/AM_Municipios_2022/AM_Municipios_2022.shp', encoding='utf-8')
    ap = gpd.read_file('../data/malhas_regionais_ibge/AP_Municipios_2022/AP_Municipios_2022.shp', encoding='utf-8')
    ma = gpd.read_file('../data/malhas_regionais_ibge/MA_Municipios_2022/MA_Municipios_2022.shp', encoding='utf-8')
    mt = gpd.read_file('../data/malhas_regionais_ibge/MT_Municipios_2022/MT_Municipios_2022.shp', encoding='utf-8')
    pa = gpd.read_file('../data/malhas_regionais_ibge/PA_Municipios_2022/PA_Municipios_2022.shp', encoding='utf-8')
    ro = gpd.read_file('../data/malhas_regionais_ibge/RO_Municipios_2022/RO_Municipios_2022.shp', encoding='utf-8')
    rr = gpd.read_file('../data/malhas_regionais_ibge/RR_Municipios_2022/RR_Municipios_2022.shp', encoding='utf-8')
    to = gpd.read_file('../data/malhas_regionais_ibge/TO_Municipios_2022/TO_Municipios_2022.shp', encoding='utf-8')

    df_states = pd.concat([ac, am, ap, ma, mt, pa, ro, rr, to])

    df_states.rename(columns={'CD_MUN':'GEOCODIBGE'}, inplace=True)

    c_units = gpd.read_file('../data/conservation_units_legal_amazon/conservation_units_legal_amazon.shp',encoding='utf-8')
    c_units.rename(columns={'nome':'UC'},inplace=True)
    
    return alerts_df, df_deter, legal_amazon, states, df_states, c_units

alerts_df, df_deter, legal_amazon, states, df_states, c_units = load_data()
folder_path = "../Visualizations/DETER/Maps"
for file in os.listdir(folder_path):
    os.remove(os.path.join(folder_path,file))



In [3]:
df_deter["UF"].unique()

array(['PA', 'MT', 'AM', 'RO', 'MA', 'TO', 'RR', 'AP', 'AC'], dtype=object)

In [4]:
dfs_deter = {uf: df_deter[df_deter['UF'] == uf] for uf in df_deter['UF'].unique()}

df_deter_PA = dfs_deter['PA']
df_deter_MT = dfs_deter['MT']
df_deter_AM = dfs_deter['AM']
df_deter_RO = dfs_deter['RO']
df_deter_MA = dfs_deter['MA']
df_deter_TO = dfs_deter['TO']
df_deter_RR = dfs_deter['RR']
df_deter_AP = dfs_deter['AP']
df_deter_AC = dfs_deter['AC']

deter_states = ['PA', 'MT', 'AM', 'RO', 'MA', 'TO', 'RR', 'AP', 'AC']
for s in deter_states:
    var_name = f'df_deter_{s}'  # Dynamically create the variable name
    grouped = globals()[var_name].groupby('VIEW_DATE', as_index=False)['AREAMUNKM'].sum()
    globals()[var_name] = pd.merge(globals()[var_name], grouped, on='VIEW_DATE', suffixes=('', '_SUM'))
    globals()[var_name] = globals()[var_name].drop_duplicates(subset=["VIEW_DATE"], keep="first")
    
print(df_deter_MA.count())
print(df_deter_MA["VIEW_DATE"].unique())



CLASSNAME        519
VIEW_DATE        519
AREAUCKM         519
UC                26
AREAMUNKM        519
MUNICIPALI       519
GEOCODIBGE       519
UF               519
ANO              519
MES              519
MES/ANO          519
AREAMUNKM_SUM    519
dtype: int64
<DatetimeArray>
['2018-01-21 00:00:00', '2018-02-16 00:00:00', '2018-01-30 00:00:00',
 '2018-02-19 00:00:00', '2018-02-22 00:00:00', '2018-02-27 00:00:00',
 '2016-11-27 00:00:00', '2018-02-28 00:00:00', '2018-03-03 00:00:00',
 '2018-05-23 00:00:00',
 ...
 '2018-05-13 00:00:00', '2018-05-14 00:00:00', '2024-09-27 00:00:00',
 '2018-01-04 00:00:00', '2018-05-28 00:00:00', '2018-04-06 00:00:00',
 '2018-02-11 00:00:00', '2018-04-15 00:00:00', '2018-01-09 00:00:00',
 '2018-01-01 00:00:00']
Length: 519, dtype: datetime64[ms]


In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# import matplotlib.pyplot as plt
# import setuptools.dist

# # Prepare the data
# df_deter_PA.reset_index(drop=True, inplace=True)

# # Function to create sequences for LSTM
# def create_sequences(data, seq_length=12):
#     X, y = [], []
#     for i in range(len(data) - seq_length):
#         X.append(data[i:i+seq_length])
#         y.append(data[i+seq_length])
#     return np.array(X), np.array(y)

# # Parameters
# seq_length = 12  # Using 12 months for yearly prediction
# future_predictions = []

# # Group by MUNICIPALI
# for municipali, group_data in df_deter_PA.groupby('MUNICIPALI'):
#     # Selecting relevant column for forecasting
#     data = group_data[['AREAMUNKM']].values

#     # Scaling the data
#     scaler = MinMaxScaler()
#     data_scaled = scaler.fit_transform(data)

#     # Creating sequences
#     X, y = create_sequences(data_scaled, seq_length)

#     if len(X) == 0:  # Skip municipalities with insufficient data
#         continue

#     # Splitting the data into training and testing sets
#     train_size = int(len(X) * 0.8)
#     X_train, y_train = X[:train_size], y[:train_size]
#     X_test, y_test = X[train_size:], y[train_size:]

#     model = Sequential([
#         LSTM(100, activation='relu', input_shape=(seq_length, 1), return_sequences=True),
#         Dropout(0.2),
#         LSTM(100, activation='relu', return_sequences=True),
#         Dropout(0.2),
#         LSTM(50, activation='relu', return_sequences=False),
#         Dropout(0.2),
#         Dense(1)
#     ])

#     model.compile(optimizer='adam', loss='mse')

#     # Training the model
#     model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1)

#     # Predicting the next year's data
#     predicted = []
#     last_sequence = data_scaled[-seq_length:]  # Last 12 months as input for next year's prediction

#     for _ in range(12):
#         next_value = model.predict(last_sequence[np.newaxis, :, :], verbose=0)
#         predicted.append(next_value[0, 0])
#         last_sequence = np.append(last_sequence[1:], next_value, axis=0)

#     # Inverse scaling to get actual values
#     predicted = scaler.inverse_transform(np.array(predicted).reshape(-1, 1)).flatten()

#     # Preparing the resulting DataFrame for the municipality
#     future_dates = pd.date_range(start="2025-01", periods=12, freq='M')
#     future_df = pd.DataFrame({
#         'CLASSNAME': ['PREDICTED'] * 12,
#         'VIEW_DATE': future_dates,
#         'AREAUCKM': [0.0] * 12,
#         'UC': [None] * 12,
#         'AREAMUNKM': predicted,
#         'MUNICIPALI': [municipali] * 12,
#         'GEOCODIBGE': [group_data['GEOCODIBGE'].iloc[0]] * 12,
#         'UF': [group_data['UF'].iloc[0]] * 12,
#         'ANO': [2025] * 12,
#         'MES': list(range(1, 13)),
#         'MES/ANO': future_dates.strftime('%Y-%m')
#     })

#     future_predictions.append(future_df)

# # Combine all predictions into a single DataFrame
# final_predictions = pd.concat(future_predictions, ignore_index=True)


2024-11-18 14:58:34.090334: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-18 14:58:34.102322: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 14:58:34.206302: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 14:58:34.306381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731959914.407950  350652 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731959914.43

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.1111
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1081
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1043
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0977
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0949
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0888
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0844
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0821
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0781
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0808
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0786
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.0848
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0825
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0759
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0724
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━━━━━

/tmp/ipykernel_350652/3175462150.py:77: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start="2025-01", periods=12, freq='M')
/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0179
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0086
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0078
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0067
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0101
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0107
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0065
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0103
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0071
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0089
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0085
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0074
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0094
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0097
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0124
Epoch 16/20
10/

/tmp/ipykernel_350652/3175462150.py:77: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start="2025-01", periods=12, freq='M')
/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.2656
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2585
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2481
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.2428
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2410
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2161
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.2096
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1945
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.1875
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1682
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.1479
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1259
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1247
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0841
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0772
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/st

/tmp/ipykernel_350652/3175462150.py:77: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start="2025-01", periods=12, freq='M')
/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.0739
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0694
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0657
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0598
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0573
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0530
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0435
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0480
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0330
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0266
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0313
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0125
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0177
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0137
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0144
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/st

/tmp/ipykernel_350652/3175462150.py:77: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start="2025-01", periods=12, freq='M')
/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-11-18 15:00:06.839610: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when

ValueError: math domain error

In [ ]:
print(alerts_df)

                FID             CLASSNAME QUADRANT PATH_ROW  VIEW_DATE SENSOR  \
0       100002_hist  CICATRIZ_DE_QUEIMADA     None   170105 2018-01-11   AWFI   
1       100003_hist  CICATRIZ_DE_QUEIMADA     None   169105 2018-01-14   AWFI   
2       100005_curr  CICATRIZ_DE_QUEIMADA     None   037017 2024-09-26    WFI   
3       100005_hist      DESMATAMENTO_VEG     None   169105 2018-01-14   AWFI   
4       100006_hist  CICATRIZ_DE_QUEIMADA     None   169105 2018-01-14   AWFI   
...             ...                   ...      ...      ...        ...    ...   
392804    9999_curr       DESMATAMENTO_CR     None   038016 2023-09-29    WFI   
392805     999_curr  CICATRIZ_DE_QUEIMADA     None   036016 2023-08-16    WFI   
392806      99_hist            DEGRADACAO        D   321074 2016-08-02  AWIFS   
392807       9_curr       DESMATAMENTO_CR     None   036016 2023-08-01    WFI   
392808       9_hist  CICATRIZ_DE_QUEIMADA        B   324078 2016-08-17  AWIFS   

            SATELLITE  AREA

In [ ]:
print("DF Deter", df_deter)
print("\nLegal Amazon", legal_amazon)
print("\nStates", states)

DF Deter                    CLASSNAME  VIEW_DATE  AREAUCKM    UC  AREAMUNKM  \
0       CICATRIZ_DE_QUEIMADA 2018-01-11       0.0  None   0.459839   
1       CICATRIZ_DE_QUEIMADA 2018-01-14       0.0  None   0.340975   
2       CICATRIZ_DE_QUEIMADA 2024-09-26       0.0  None   1.373554   
3           DESMATAMENTO_VEG 2018-01-14       0.0  None   0.070781   
4       CICATRIZ_DE_QUEIMADA 2018-01-14       0.0  None   0.149432   
...                      ...        ...       ...   ...        ...   
392804       DESMATAMENTO_CR 2023-09-29       0.0  None   0.206886   
392805  CICATRIZ_DE_QUEIMADA 2023-08-16       0.0  None   0.425538   
392806            DEGRADACAO 2016-08-02       0.0  None  22.043149   
392807       DESMATAMENTO_CR 2023-08-01       0.0  None   0.139690   
392808  CICATRIZ_DE_QUEIMADA 2016-08-17       0.0  None   7.582060   

                MUNICIPALI GEOCODIBGE  UF   ANO  MES  MES/ANO  
0             Monte Alegre    1504802  PA  2018    1  2018-01  
1                 Itai

In [ ]:
print("DF Municipalities", df_states)
print("\nC Units", c_units)

DF Municipalities     GEOCODIBGE          NM_MUN SIGLA_UF  AREA_KM2  \
0      1200013      Acrelândia       AC  1811.613   
1      1200054    Assis Brasil       AC  4979.073   
2      1200104       Brasiléia       AC  3928.174   
3      1200138          Bujari       AC  3034.869   
4      1200179        Capixaba       AC  1705.824   
..         ...             ...      ...       ...   
134    1721208  Tocantinópolis       TO  1083.600   
135    1721257        Tupirama       TO   706.883   
136    1721307      Tupiratins       TO   889.126   
137    1722081    Wanderlândia       TO  1365.431   
138    1722107         Xambioá       TO  1190.489   

                                              geometry  
0    POLYGON ((-67.07612 -10.08798, -67.07659 -10.0...  
1    POLYGON ((-69.55253 -10.87353, -69.52086 -10.8...  
2    POLYGON ((-68.75712 -11.01097, -68.75752 -11.0...  
3    POLYGON ((-67.92167 -9.69355, -67.91736 -9.693...  
4    POLYGON ((-67.73403 -10.71177, -67.73414 -10.7...  
.. 

In [7]:
# LOADING TEXTS (ENGLISH AND PORTUGUESE)

df_texts = pd.read_csv('../texts/texts_deter.csv', sep='§', engine='python')
english = {list(df_texts['Key'])[i]: list(df_texts['English'])[i] for i in range(len(list(df_texts['Key'])))}
# portuguese = {list(df_texts['Key'])[i]: list(df_texts['Portuguese'])[i] for i in range(len(list(df_texts['Key'])))}

classes_deter_en = {'CICATRIZ_DE_QUEIMADA': 'Forest Fire Scar',
          'DESMATAMENTO_CR': 'Deforestation with Exposed Soil',
          'DESMATAMENTO_VEG': 'Deforestation with Vegetation',
          'MINERACAO': 'Mining',
          'DEGRADACAO': 'Degradation',
          'CS_DESORDENADO': 'Selective Logging Type 1 (Disordered)',
          'CS_GEOMETRICO': 'Selective Logging Type 2 (Geometric)',
}

states_dict = {
    "MT": "Mato Grosso",
    "PA": "Pará",
    "AM": "Amazonas",
    "RO": "Rondônia",
    "MA": "Maranhão",
    "RR": "Roraima",
    "AC": "Acre",
    "TO": "Tocantins",
    "AP": "Amapá"
}

def get_texts():
    return classes_deter_en, english



dict_classes, texts = get_texts()

In [8]:
print(dict_classes)

{'CICATRIZ_DE_QUEIMADA': 'Forest Fire Scar', 'DESMATAMENTO_CR': 'Deforestation with Exposed Soil', 'DESMATAMENTO_VEG': 'Deforestation with Vegetation', 'MINERACAO': 'Mining', 'DEGRADACAO': 'Degradation', 'CS_DESORDENADO': 'Selective Logging Type 1 (Disordered)', 'CS_GEOMETRICO': 'Selective Logging Type 2 (Geometric)'}


In [9]:
print(texts)

{'page_title': 'Amazon Deforestation Monitor', 'date_format': '%m-%d-%Y', 'date_format2': '%m-%Y', 'date_format3': 'MM/DD/YYYY', 'deter_expander_title': 'What is DETER?', 'deter_expander_desc_1': 'The [DETER](http://www.inpe.br/cra/projetos_pesquisas/deter.php) system, short for "Real-Time Deforestation Detection", is a system developed by the Brazilian National Institute for Space Research (INPE), aimed at monitoring and identifying changes in forest cover within the Legal Amazon in Brazil. This system is essential for environmental enforcement and combating illegal deforestation, providing crucial data for prevention and control actions.\nThrough high-resolution satellite images, DETER can detect areas of deforestation and forest degradation with high precision and almost in real-time. This information is made available periodically, allowing for a swift response from environmental authorities.\n\nDETER\'s detections are classified into various categories, reflecting different types 

In [10]:
def get_centroids(geo_df, mode=2, crs='EPSG:31982'):

    if mode==0:
        centroids = geo_df.copy()
        centroids["centroid"] = centroids.geometry.centroid
        centroids["latitude"] = centroids.centroid.y
        centroids["longitude"] = centroids.centroid.x
        return centroids

    if mode==1:
        geo_df_proj = geo_df.copy()
        geo_df_proj.to_crs(crs)
        
        geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
        geo_df_proj['latitude'] = geo_df_proj.centroid.y
        geo_df_proj['longitude'] = geo_df_proj.centroid.x
        
        centroids = gpd.GeoDataFrame(geo_df_proj, geometry='centroid', crs=crs)
    
        centroids = centroids.to_crs(geo_df.crs)
        return centroids

    if mode==2:
        df = geo_df.copy()
        df['representative_point'] = df.geometry.representative_point()
        df['latitude'] = df['representative_point'].apply(lambda p: p.y)
        df['longitude'] = df['representative_point'].apply(lambda p: p.x)
        return df

def folium_map_init():
    map = folium.Map(location=[-7.25, -60], zoom_start=4)
    return map

def folium_add_markers(container, df_data, geo_df, get_centroid_mode, df_deter, key, popup_title_column, popup_total_area_text='Área Total Afetada:',total_area_column='AREAMUNKM'):

    all_classes = sorted(df_deter['CLASSNAME'].unique())
    df_centroids = get_centroids(geo_df,get_centroid_mode)
    
    for idx, row in df_data.iterrows():

        coords = df_centroids.loc[df_centroids[key] == row[key]].iloc[0]

        info = ''
        
        if row['AREAMUNKM']>0:
            df_stats = df_deter[df_deter[key] == row[key]]
            df_stats_summed = df_stats.groupby('CLASSNAME')['AREAMUNKM'].sum().reset_index()
            df_stats_complete = pd.DataFrame({'CLASSNAME': all_classes})
            df_stats_complete = df_stats_complete.merge(df_stats_summed, on='CLASSNAME', how='left').fillna(0)
            df_stats_complete['DESC'] = df_stats_complete['CLASSNAME'].map(dict_classes)
            df_stats_complete = df_stats_complete.sort_values(by='AREAMUNKM', ascending=False)
    
            total = df_stats_complete['AREAMUNKM'].sum()
            
            # Calculates percentage of every class
            for ind, lin in df_stats_complete.iterrows():
                perc = (lin['AREAMUNKM'] * 100) / total
                info += f"{lin['DESC']}: {lin['AREAMUNKM']:.0f} km² ({perc:.2f}%)<br>"
        
        popup_text = f"""
        <div style='white-space: nowrap;'>
        <span style='font-size: 16px; font-weight: bold;'>{row[popup_title_column]}</span><br><br> {popup_total_area_text} {row[total_area_column]:.0f} km²<br><br> {info}
        </div>
        """

        # Add marker on Map or MarkerCluster (container)
        folium.Marker(
            location=[coords['latitude'], coords['longitude']],
            popup=popup_text,
            icon=folium.Icon(color='red', icon='triangle-exclamation', prefix='fa')
        ).add_to(container)
    
    return container

In [11]:
def save_map(file_name,map):
    map.save(f"../Visualizations/DETER/Maps/{file_name}.html")

### States Map:

In [12]:
def states_map():
    ############# Data Preparation #############
    # Load shapefiles for regions (if not already loaded)
    # legal_amazon = gpd.read_file('data/brazilian_legal_amazon/brazilian_legal_amazon.shp', encoding='utf-8')
    # states = gpd.read_file('data/states_legal_amazon/states_legal_amazon.shp', encoding='utf-8')
    
    df_deter = alerts_df.copy()
    gb_uf = df_deter.groupby('UF')['AREAMUNKM'].sum().sort_values(ascending=False)
    gb_uf = pd.DataFrame(gb_uf)
    gb_uf['NOME_ESTADO'] = gb_uf.index.map(states_dict)
    gb_uf['NOME_SIGLA'] = gb_uf['NOME_ESTADO'] + ' (' + gb_uf.index + ')' 
    gb_uf = gb_uf.reset_index()

    states_copy = states.copy()
    states_copy = states_copy.rename(columns={'sigla': 'UF'})

    #############       Folium       #############
    map = folium_map_init()

    # Customizing state borders color
    style_states = {'fillOpacity': 0.3, 'color': '#005f73', 'weight': 2}
    folium.GeoJson(states_copy, name='States', style_function=lambda x: style_states).add_to(map)

    # Customizing Legal Amazon border color
    style_legal_amazon = {'fillOpacity': 0, 'color': '#0a9396', 'weight': 3}
    folium.GeoJson(legal_amazon, name='Legal Amazon', style_function=lambda x: style_legal_amazon).add_to(map)

    # Customizing Choropleth color scheme
    folium.Choropleth(
        geo_data=states_copy,
        data=gb_uf,
        columns=['UF', 'AREAMUNKM'],
        key_on='feature.properties.UF',
        fill_color='Reds',
        fill_opacity=0.7,
        line_opacity=0.2,
        nan_fill_color='white',
        bins=8,
        highlight=True,
        legend_name='Affected Area in km²',
        name='Most Affected States'
    ).add_to(map)

    # Adding markers with popups for more information
    map = folium_add_markers(map, gb_uf, states_copy, 1, df_deter, 'UF', 'NOME_SIGLA', texts['total_dmg'], 'AREAMUNKM')

    # Adding layer control
    folium.LayerControl().add_to(map)

    return map


In [13]:
dict_classes, texts = get_texts()
map = states_map()
save_map('States_EN',map)

/tmp/ipykernel_350652/1685801861.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
/tmp/ipykernel_350652/1685801861.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['latitude'] = geo_df_proj.centroid.y
/tmp/ipykernel_350652/1685801861.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['longitude'] = geo_df_proj.centroid.x


### Cities Map

In [14]:
def cities_map(filter=[]):
    
    ############# Data Preparation #############

    geocodibge = alerts_df.drop_duplicates(subset='MUNICIPALI').set_index('MUNICIPALI')['GEOCODIBGE']
    sum_areamunkm = alerts_df.groupby('MUNICIPALI')['AREAMUNKM'].sum().reset_index()
    sum_areamunkm['GEOCODIBGE'] = sum_areamunkm['MUNICIPALI'].map(geocodibge)

    merge = pd.merge(df_states, sum_areamunkm, on='GEOCODIBGE', how='left')

    if len(filter)>0:
        merge = merge[merge['SIGLA_UF'].isin(filter)]
    
    # merge[merge['AREAMUNKM'].isna()]
    # 226 cidades não contém avisos. Esses valores ausentes serão preenchidos com 0.
    merge['AREAMUNKM'].fillna(0, inplace=True)

    #############       Folium       #############
    map = folium_map_init()

    style_cities = {'fillOpacity':0 ,'color' : '#117306', 'weight': 1}
    folium.GeoJson(merge, name = 'Cities', style_function= lambda x: style_cities).add_to(map)

    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states, name = 'States', style_function= lambda x: style_states).add_to(map)
    
    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)
    
    folium.Choropleth(geo_data=merge.to_json(),
                  name='Choropleth',
                  data=merge,
                  columns=['GEOCODIBGE', 'AREAMUNKM'],
                  key_on = 'feature.properties.GEOCODIBGE',
                  fill_color = 'Reds',
                  nan_fill_color = 'white',
                  highlight = True,
                  legend_name='Affected Area in km²').add_to(map)

    marker_cluster = MarkerCluster().add_to(map)
    marker_cluster = folium_add_markers(marker_cluster, merge, merge, 2, alerts_df, 'GEOCODIBGE', 'NM_MUN', texts['total_dmg'], 'AREAMUNKM')
    folium.LayerControl().add_to(map)

    return map

In [15]:
lst_states = list(df_states['SIGLA_UF'].unique())
lst_states

['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']

In [16]:
dict_classes, texts = get_texts()

for i in range(len(lst_states)):
    filter = []
    filter.append(lst_states[i])
    map = cities_map(filter)
    map_name = 'Cities_EN_' + filter[0]
    save_map(map_name,map)
    print(map_name + ' saved.')

/tmp/ipykernel_350652/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_AC saved.


/tmp/ipykernel_350652/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_AM saved.


/tmp/ipykernel_350652/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


...

Cities_EN_RO saved.


/tmp/ipykernel_350652/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_RR saved.


/tmp/ipykernel_350652/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_TO saved.


In [17]:
dict_classes, texts = get_texts()
map = cities_map([])
map_name = 'All_Cities_EN'
save_map(map_name,map)
print(map_name + ' saved.')

/tmp/ipykernel_350652/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


All_Cities_EN saved.


### C Units

In [18]:
def normalize_string(s):
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^\w\s]', '', s)
    s = s.upper()
    return s

def c_units_map():
    ############# Data Preparation #############
    c_units_copy = c_units.copy()
    c_units_copy['UC'] = c_units_copy['UC'].apply(normalize_string)
    
    alerts_uc = alerts_df[alerts_df['UC'].notna()].copy()
    dic_correcao = {'FLORESTA NACIONAL DE ALTAMIRA': 'FLORESTA NACIONAL ALTAMIRA', 
                    'FLORESTA NACIONAL DE CAXIUANÂ': 'FLORESTA NACIONAL DE CAXIUANÃ', 
                    'FLORESTA NACIONAL DO AMANA': 'FLORESTA NACIONAL DO AMANÁ',
                    'FLORESTA NACIONAL DO BOM FUTURO': 'FLORESTA NACIONAL DE BOM FUTURO',
                    'FLORESTA NACIONAL DO ITACAIUNAS': 'FLORESTA NACIONAL DE ITACAIUNAS',
                    'FLORESTA NACIONAL DO JATUARANA': 'FLORESTA NACIONAL DE JATUARANA',
                    'FLORESTA NACIONAL DO PURUS': 'RESERVA EXTRATIVISTA DO MÉDIO PURÚS',
                    'FLORESTA NACIONAL DO TAPAJÓS': 'FLORESTA NACIONAL DE TAPAJÓS',
                    'FLORESTA NACIONAL DO TAPIRAPÉAQUIRI': 'FLORESTA NACIONAL DE TAPIRAPÉAQUIRI',
                    'FLORESTA NACIONAL MAPIÁ  INAUINI': 'FLORESTA NACIONAL DE MAPIÁINAUINÍ',
                    'PARQUE NACIONAL SERRA DA CUTIA': 'PARQUE NACIONAL DA SERRA DA CUTIA',
                    'RESERVA BIOLÓGICA NASCENTES DA SERRA DO CACHIMBO': 'RESERVA BIOLÓGICA NASCENTES SERRA DO CACHIMBO',
                    'RESERVA EXTRATIVISTA DO ALTO JURUÁ': 'RESERVA EXTRATIVISTA ALTO JURUÁ',
                    'RESERVA EXTRATIVISTA DO ALTO TARAUACÁ': 'RESERVA EXTRATIVISTA ALTO TARAUACÁ',
                    'RESERVA EXTRATIVISTA DO BAIXO JURUÁ': 'RESERVA EXTRATIVISTA BAIXO JURUÁ',
                    'RESERVA EXTRATIVISTA DO CIRIACO': 'RESERVA EXTRATIVISTA DO CIRIÁCO',
                    'RESERVA EXTRATIVISTA DO LAGO DO CUNIÃ': 'RESERVA EXTRATIVISTA LAGO DO CUNIÃ',
                    'RESERVA EXTRATIVISTA DO MÉDIO JURUÁ': 'RESERVA EXTRATIVISTA MÉDIO JURUÁ',
                    'RESERVA EXTRATIVISTA DO RIO CAJARI': 'RESERVA EXTRATIVISTA RIO CAJARI',
                    'RESERVA EXTRATIVISTA DO RIO DO CAUTÁRIO': 'RESERVA EXTRATIVISTA RIO CAUTÁRIO',
                    'RESERVA EXTRATIVISTA DO RIO OURO PRETO': 'RESERVA EXTRATIVISTA RIO OURO PRETO',
                    'RESERVA EXTRATIVISTA RIO UNINI': 'RESERVA EXTRATIVISTA DO RIO UNINI',
                    'RESERVA EXTRATIVISTA TAPAJÓSARAPIUNS': 'RESERVA EXTRATIVISTA TAPAJÓS ARAPIUNS',
                    'RESERVA EXTRATIVISTA TAPAJÓS-ARAPIUNS': 'RESERVA EXTRATIVISTA TAPAJÓS ARAPIUNS',
                    'RESERVA EXTRATIVISTA TERRA GRANDE  PRACUÚBA': 'RESERVA EXTRATIVISTA TERRA GRANDE PRACUUBA',
                    'RESERVA EXTRATIVISTA TERRA GRANDE - PRACUÚBA': 'RESERVA EXTRATIVISTA TERRA GRANDE PRACUUBA',
                    'ÁREA DE PROTEÇÃO AMBIENTAL DOS MEANDROS DO RIO ARAGUAIA': 'ÁREA DE PROTEÇÃO AMBIENTAL MEANDROS DO ARAGUAIA',
                    'ÁREA DE RELEVANTE INTERESSE ECOLÓGICO SERINGAL NOVA ESPERANÇA': 'ÁREA DE RELEVANTE INTERESSE ECOLÓGICA SERINGAL NOVA ESPERANÇA',
                    'ESTAÇÃO ECOLÓGICA JUAMI-JAPURÁ': 'ESTAÇÃO ECOLÓGICA JUAMIJAPURÁ',
                    'FLORESTA NACIONAL DE BALATA-TUFARI': 'FLORESTA NACIONAL DE BALATATUFARI',
                    'FLORESTA NACIONAL DE SARACÁ-TAQUERA': 'FLORESTA NACIONAL DE SARACÁTAQUERA',
                    'FLORESTA NACIONAL MAPIÁ - INAUINI': 'FLORESTA NACIONAL DE MAPIÁINAUINÍ',
                    'RESERVA EXTRATIVISTA AUATÍ-PARANÁ': 'RESERVA EXTRATIVISTA AUATÍPARANÁ',
                    'RESERVA EXTRATIVISTA DO CAZUMBÁ-IRACEMA': 'RESERVA EXTRATIVISTA DO CAZUMBÁIRACEMA',
                    'RESERVA EXTRATIVISTA GURUPÁ-MELGAÇO': 'RESERVA EXTRATIVISTA GURUPÁMELGAÇO',
                    'RESERVA EXTRATIVISTA IPAÚ-ANILZINHO': 'RESERVA EXTRATIVISTA IPAÚANILZINHO'}
    alerts_uc['UC'] = alerts_uc['UC'].replace(dic_correcao)
    gc_uc = alerts_uc.groupby('UC')['AREAMUNKM'].sum().reset_index()
    gc_uc['UC'] = gc_uc['UC'].replace(dic_correcao)
    

    def uc_geodf(state):
        first_alert = alerts_df[alerts_df['UC'] == state]['geometry'].iloc[0]
        representative_point = first_alert
        new_record = {
            'UC': state,
            'geometry': representative_point
        }
        return gpd.GeoDataFrame([new_record], crs=c_units_copy.crs)
    
    c_units_copy = pd.concat([c_units_copy, uc_geodf('ESTAÇÃO ECOLÓGICA DE CARACARAÍ')], ignore_index=True)
    c_units_copy = pd.concat([c_units_copy, uc_geodf('ESTAÇÃO ECOLÓGICA DE IQUÊ')], ignore_index=True)
    
    merge_ucs = pd.merge(c_units_copy, gc_uc, on='UC', how='left').fillna(0)


    
    #############       Folium       #############

    map = folium_map_init()

    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)

    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states, name = 'States', style_function= lambda x: style_states).add_to(map)

    style_ucs = {'fillOpacity':0 ,'color' : '#3d1601', 'weight': 1}
    folium.GeoJson(c_units_copy, name = 'Conservation Units', style_function= lambda x: style_ucs).add_to(map)


    folium.Choropleth(geo_data=merge_ucs.to_json(),
                  name='Choropleth',
                  data=merge_ucs,
                  columns=['UC', 'AREAMUNKM'],
                  key_on = 'feature.properties.UC',
                  fill_color = 'YlOrRd',
                  nan_fill_color = 'white',
                  highlight = True,
                  legend_name='Affected Area in km²').add_to(map)
    
    marker_cluster = MarkerCluster().add_to(map)

    marker_cluster = folium_add_markers(marker_cluster,merge_ucs,merge_ucs, 2, alerts_uc, 'UC', 'UC', texts['total_dmg'], 'AREAMUNKM')

    folium.LayerControl().add_to(map)
    return map

In [19]:
dict_classes, texts = get_texts()
map = c_units_map()
save_map('C_Units_EN',map)

LSTM

In [20]:
from tensorflow.keras.layers import Dropout, Bidirectional, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Function to build and train an improved robust LSTM model
def train_lstm(X_train, y_train, X_test, y_test):
    model = Sequential()
    
    # First LSTM layer with dropout and layer normalization
    model.add(Bidirectional(LSTM(128, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))))
    model.add(LayerNormalization())
    model.add(Dropout(0.3))
    
    # Second LSTM layer with dropout and layer normalization
    model.add(LSTM(64, activation='tanh', return_sequences=True))
    model.add(LayerNormalization())
    model.add(Dropout(0.3))
    
    # Third LSTM layer
    model.add(LSTM(32, activation='tanh'))
    model.add(Dropout(0.2))
    
    # Dense output layer
    model.add(Dense(1))
    
    # Compile the model with MSE loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)
    
    # Train the model
    model.fit(X_train, y_train, 
              epochs=100, 
              batch_size=32,
              validation_data=(X_test, y_test), 
              verbose=1, 
              callbacks=[early_stopping, reduce_lr])
    
    return model


In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Function to prepare data for LSTM
def prepare_data(df, feature, target, sequence_length=30):
    df = df.sort_values('VIEW_DATE')
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[[target]])
    
    # Create sequences
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(scaled_data[i+sequence_length])
    
    X = np.array(X)
    y = np.array(y)
    return X, y, scaler

# Function to predict and inverse transform
def predict_future(model, data, scaler, steps=10):
    predictions = []
    input_sequence = data[-1]
    
    for _ in range(steps):
        pred = model.predict(input_sequence[np.newaxis, :, :])
        predictions.append(pred[0][0])
        input_sequence = np.vstack((input_sequence[1:], pred))
    
    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Prepare and train LSTM for each state
sequence_length = 30
future_steps = 90

In [22]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize a dictionary to store model evaluation metrics
model_accuracies = {}
models = {}

# Iterate over each state to split data, train LSTM, and evaluate
for state in deter_states:
    var_name = f'df_deter_{state}'
    df = globals()[var_name]  # Access the dataframe dynamically
    df = df.sort_values('VIEW_DATE')  # Ensure data is sorted by date
    
    # Prepare the data
    X, y, scaler = prepare_data(df, feature='VIEW_DATE', target='AREAMUNKM_SUM', sequence_length=sequence_length)
    
    # Split into train and test sets
    split = int(0.8 * len(X))
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]
    
    # Train the LSTM model
    model = train_lstm(X_train, y_train, X_test, y_test)
    models[state] = model  # Save the model
    
    # Evaluate on the test set
    y_pred = model.predict(X_test)
    y_test_inversed = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inversed = scaler.inverse_transform(y_pred)
    
    # Calculate RMSE as the evaluation metric
    rmse = np.sqrt(mean_squared_error(y_test_inversed, y_pred_inversed))
    model_accuracies[state] = rmse

# Display model accuracies for each state
import pandas as pd
accuracy_df = pd.DataFrame.from_dict(model_accuracies, orient='index', columns=['RMSE'])
accuracy_df.index.name = 'State'
accuracy_df.reset_index(inplace=True)



/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - loss: 0.2405 - val_loss: 0.0076 - learning_rate: 0.0010
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0116 - val_loss: 0.0029 - learning_rate: 0.0010
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0068 - val_loss: 0.0030 - learning_rate: 0.0010
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0046 - val_loss: 0.0029 - learning_rate: 0.0010
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0039 - val_loss: 0.0024 - learning_rate: 0.0010
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0026 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0019 - val_loss: 0.0026 - learning_rate: 0.0010
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0028 - val_loss: 0.0026 - learning_rate: 0.0010
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0016 - val_loss: 0.0026 - learning_rate: 5.0000e-04
Epoch 

/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 0.3413 - val_loss: 0.0050 - learning_rate: 0.0010
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0144 - val_loss: 0.0055 - learning_rate: 0.0010
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0101 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0089 - val_loss: 0.0047 - learning_rate: 0.0010
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0069 - val_loss: 0.0047 - learning_rate: 0.0010
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0067 - val_loss: 0.0047 - learning_rate: 0.0010
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0059 - val_loss: 0.0045 - learning_rate: 5.0000e-04
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0054 - val_loss: 0.0048 - learning_rate: 5.0000e-04
Epoch 9/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0062 - val_loss: 0.0047 - learning_rate: 5.0000e-04
Epoch 10/1

/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - loss: 0.3836 - val_loss: 0.0237 - learning_rate: 0.0010
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0713 - val_loss: 0.0110 - learning_rate: 0.0010
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0323 - val_loss: 0.0113 - learning_rate: 0.0010
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0188 - val_loss: 0.0109 - learning_rate: 0.0010
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0111 - val_loss: 0.0105 - learning_rate: 0.0010
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0091 - val_loss: 0.0104 - learning_rate: 0.0010
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0091 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0062 - val_loss: 0.0093 - learning_rate: 0.0010
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0049 - val_loss: 0.0099 - learning_rate: 0.0010
Epoch 10/100
11/11 ━━

/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 177ms/step - loss: 1.2433 - val_loss: 0.1070 - learning_rate: 0.0010
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.1237 - val_loss: 0.0501 - learning_rate: 0.0010
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0728 - val_loss: 0.0860 - learning_rate: 0.0010
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0868 - val_loss: 0.0305 - learning_rate: 0.0010
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0395 - val_loss: 0.0415 - learning_rate: 0.0010
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0330 - val_loss: 0.0349 - learning_rate: 0.0010
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0471 - val_loss: 0.0350 - learning_rate: 0.0010
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0293 - val_loss: 0.0292 - learning_rate: 5.0000e-04
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0297 - val_loss: 0.0302 - learning_rate: 5.0000e-04
Epoch 10/100
5/5 ━━━━━━━━━━━━━━

/home/prathiba/pra/GeorgiaTech/Fall24/CSE6242/Data_Analysis_Project/.environment/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - loss: 0.2067 - val_loss: 0.0091 - learning_rate: 0.0010
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0372 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0219 - val_loss: 0.0098 - learning_rate: 0.0010
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0197 - val_loss: 0.0095 - learning_rate: 0.0010
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0133 - val_loss: 0.0100 - learning_rate: 5.0000e-04
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0119 - val_loss: 0.0093 - learning_rate: 5.0000e-04
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step


In [23]:
import matplotlib.pyplot as plt

future_dataframes = {} 

# Plot train, test, and forecasted data for each state starting from 2023 as simple line plots
for state in deter_states:
    var_name = f'df_deter_{state}'
    df = globals()[var_name]  # Access the dataframe dynamically
    df = df.sort_values('VIEW_DATE')  # Ensure data is sorted by date

    # Filter data for dates from 2023 onward
    df = df[df['VIEW_DATE'] >= '2023-01-01']

    # Prepare the data
    X, y, scaler = prepare_data(df, feature='VIEW_DATE', target='AREAMUNKM_SUM', sequence_length=sequence_length)
    
    # Split into train and test sets
    split = int(0.8 * len(X))
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]
    
    # Adjust corresponding dates for train and test sets
    train_dates = df['VIEW_DATE'].iloc[sequence_length:split + sequence_length].values  # Match the train set
    test_dates = df['VIEW_DATE'].iloc[split + sequence_length:].values  # Match the test set
    
    # Get the corresponding model
    model = models[state]
    
    # Make predictions for test set
    y_pred = model.predict(X_test)
    y_test_inversed = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inversed = scaler.inverse_transform(y_pred)
    
    # Generate future predictions
    future_preds = predict_future(model, X, scaler, steps=future_steps)
    future_dates = pd.date_range(start=test_dates[-1], periods=future_steps + 1, freq='D')[1:]  # Generate future dates

    # Scale back to original values for visualization
    y_train_inversed = scaler.inverse_transform(y_train.reshape(-1, 1))
    
    future_df = pd.DataFrame({
        'Future_Dates': future_dates,
        'Future_Preds': future_preds.flatten()  # Flatten to ensure it's a 1D array
    })
    future_dataframes[state] = future_df 
    
    # Plot train, test, and forecasted sets
    # plt.figure(figsize=(12, 6))
    # plt.plot(train_dates, y_train_inversed, label='Train Data', linestyle='-')
    # plt.plot(test_dates, y_test_inversed, label='Test Data', linestyle='--')
    # plt.plot(test_dates, y_pred_inversed, label='Forecasted Test Data', linestyle=':')
    # plt.plot(future_dates, future_preds, label='Future Predictions', linestyle='-.')
    # plt.title(f"Train, Test, and Forecast for {state} (AREAMUNKM_SUM) - From 2023")
    # plt.xlabel("Date")
    # plt.ylabel("AREAMUNKM_SUM")
    # plt.xticks(rotation=45)
    # plt.legend()
    # plt.grid()
    # plt.tight_layout()
    # plt.show()


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

In [24]:
combined_df = pd.concat(
    [df.assign(State=state) for state, df in future_dataframes.items()],
    ignore_index=True
)

combined_df.rename(columns={"Future_Dates": "future_dates", "Future_Preds": "future_preds", "State": "state"}, inplace=True)

combined_df.to_csv('future_preds.csv', index=False)

# Convert 'future_dates' to datetime for proper grouping
combined_df['future_dates'] = pd.to_datetime(combined_df['future_dates'])

# Add month and year columns for grouping
combined_df['year'] = combined_df['future_dates'].dt.year
combined_df['month'] = combined_df['future_dates'].dt.month

# Group by 'state', 'year', and 'month', then aggregate 'future_preds' using sum
grouped_df = combined_df.groupby(['state', 'year', 'month'])['future_preds'].sum().reset_index()

In [25]:
print(grouped_df)

   state  year  month  future_preds
0     AC  2024      9     46.463341
1     AC  2024     10    126.928574
2     AC  2024     11    114.367500
3     AC  2024     12     90.145096
4     AM  2024      9     82.450439
5     AM  2024     10    592.160339
6     AM  2024     11    480.530701
7     AM  2024     12    396.440582
8     AP  2024      9     13.755094
9     AP  2024     10     31.159214
10    AP  2024     11     13.870485
11    AP  2024     12     11.093808
12    MA  2024      9      3.462486
13    MA  2024     10     34.051239
14    MA  2024     11     36.298050
15    MA  2024     12     31.393208
16    MT  2024      9    488.679932
17    MT  2024     10   3356.899170
18    MT  2024     11   2170.499023
19    MT  2024     12   1791.354858
20    PA  2024      9    165.511459
21    PA  2024     10   1368.062622
22    PA  2024     11    658.630127
23    PA  2024     12    521.740723
24    RO  2024      9    103.574051
25    RO  2024     10    512.137451
26    RO  2024     11    384

TEST CODE

In [ ]:
def states_map_pred():
    """
    Generates a Folium map for a specific month and year with choropleth and markers.
    Assumes the necessary datasets (`states`, `grouped_df`, `df_deter`) are globally available.
    
    Returns:
        folium.Map: A map with a choropleth and markers.
    """
    ############# Data Preparation #############
    # Define month and year for the map
    selected_month = 10
    selected_year = 2024

    # Filter grouped_df for the selected month and year
    filtered_data = grouped_df[(grouped_df['month'] == selected_month) & (grouped_df['year'] == selected_year)]

    if filtered_data.empty:
        raise ValueError(f"No data available for {selected_year}-{selected_month:02d} in grouped_df.")

    # Merge filtered_data with states GeoDataFrame
    states_copy = states.copy()
    states_copy = states_copy.rename(columns={'sigla': 'UF'})  # Rename for consistency
    map_data = states_copy.merge(filtered_data, left_on='UF', right_on='state', how='left')
    map_data.drop(columns = ["state"],inplace = True)
    
    print(map_data)

    ############# Folium Map Initialization #############
    # Initialize the map
    map = folium_map_init()

    # Add state borders
    style_states = {'fillOpacity': 0.3, 'color': '#005f73', 'weight': 2}
    folium.GeoJson(map_data, name='States', style_function=lambda x: style_states).add_to(map)

    # Add choropleth layer
    folium.Choropleth(
        geo_data=map_data,
        data=filtered_data,
        columns=['UF', 'future_preds'],
        key_on='feature.properties.UF',
        fill_color='Reds',
        fill_opacity=0.7,
        line_opacity=0.2,
        nan_fill_color='white',
        bins=8,
        legend_name=f"Future Predictions ({selected_year}-{selected_month:02d})",
        name='Future Predictions'
    ).add_to(map)

    # Add markers with detailed popups
    map = folium_add_markers(
        container=map,
        df_data=filtered_data,
        geo_df=states_copy,
        get_centroid_mode=1,
        df_deter=df_deter,
        key='UF',  # Use 'UF' as the key from states_copy
        popup_title_column='state',  # Use 'state' from grouped_df for popups
        popup_total_area_text='Future Predictions:',
        total_area_column='future_preds'
    )

    # Add layer control
    folium.LayerControl().add_to(map)

    return map

In [ ]:
import pandas as pd

# Initialize an empty dataframe to store the merged predictions
merged_predictions = pd.DataFrame()

# Iterate over each state to merge predicted data
for state in deter_states:
    # var_name = f'df_deter_{state}'
    # df = globals()[var_name]  # Access the dataframe dynamically
    # df = df.sort_values('VIEW_DATE')  # Ensure data is sorted by date

    # # Filter data for dates from 2023 onward
    # df_filtered = df[df['VIEW_DATE'] >= '2023-01-01']
    
    # # Prepare the data
    # X, y, scaler = prepare_data(df_filtered, feature='VIEW_DATE', target='AREAMUNKM_SUM', sequence_length=sequence_length)
    
    # # Get the corresponding model
    # model = models[state]
    
    # # Make predictions for test set
    # y_pred = model.predict(X)
    # y_pred_inversed = scaler.inverse_transform(y_pred)
    pred_start_idx = sequence_length  # The first prediction index
    pred_dates = df['VIEW_DATE'].iloc[pred_start_idx:pred_start_idx + len(y_pred_inversed)].values  # Match dates with predictions
    
    # Create a dataframe for predictions
    pred_df = pd.DataFrame({
        'DATE': pred_dates,
        f'PREDICTED_{state}_AREAMUNKM_SUM': y_pred_inversed.flatten()
    })
    
    # Merge with the main dataframe based on the date
    if merged_predictions.empty:
        merged_predictions = pred_df
    else:
        merged_predictions = pd.merge(merged_predictions, pred_df, on='DATE', how='outer')
# Optionally, save it to a CSV file for further use
merged_predictions.to_csv('merged_predictions.csv', index=False)

